In [1]:
import datetime as dt
import logging

import httpx
import shapely
import sqlmodel

from arpav_ppcv import (
    config,
    database,
    operations,
)
from arpav_ppcv.schemas.base import (
    CoverageDataSmoothingStrategy,
    ObservationDataSmoothingStrategy,
)

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)

http_client = httpx.Client()
settings = config.get_settings()
session = sqlmodel.Session(database.get_engine(settings))

forecast_coverage = database.get_forecast_coverage(
    session, "forecast-tas-absolute-annual-arpa_vfvg-model_ensemble-rcp26-winter")

location = shapely.Point(12.0721, 46.1415)  # ~300m close to station 'arpa_v-25'
temporal_range = (None, None)

In [3]:
cov_series = operations._get_forecast_coverage_coverage_time_series(
    thredds_settings=settings.thredds_server,
    session=session,
    http_client=http_client, 
    forecast_coverage=forecast_coverage, 
    point_geom=location,
    smoothing_strategies=[
        CoverageDataSmoothingStrategy.NO_SMOOTHING,
        CoverageDataSmoothingStrategy.MOVING_AVERAGE_11_YEARS,
    ], 
    include_uncertainty=True,
    include_related_models=True,
    temporal_range=temporal_range,
)

DEBUG:arpav_ppcv.thredds.ncss:identifier='forecast-tas-absolute-annual-arpa_vfvg-model_ensemble-rcp26-winter-upper_uncertainty-t40u9vxjnfs1-*-*-no_smoothing'
DEBUG:arpav_ppcv.thredds.ncss:ncss_url='http://thredds:8080/thredds/ncss/grid/ensymbc/std/clipped/tas_stdup_rcp26_DJF_ts19762100_ls_VFVG.nc'
DEBUG:arpav_ppcv.thredds.ncss:netcdf_variable_name='tas_stdup'
DEBUG:arpav_ppcv.database:Adding 'forecast-tas-absolute-annual-arpa_vfvg-ec_earth_cclm_4_8_17-rcp26-winter' to the result...
DEBUG:arpav_ppcv.database:Adding 'forecast-tas-absolute-annual-arpa_vfvg-ec_earth_racmo22e-rcp26-winter' to the result...
DEBUG:arpav_ppcv.database:Adding 'forecast-tas-absolute-annual-arpa_vfvg-ec_earth_rca4-rcp26-winter' to the result...
DEBUG:arpav_ppcv.database:Adding 'forecast-tas-absolute-annual-arpa_vfvg-hadgem2_racmo22e-rcp26-winter' to the result...
DEBUG:arpav_ppcv.database:Adding 'forecast-tas-absolute-annual-arpa_vfvg-mpi_esm_lr_remo2009-rcp26-winter' to the result...
DEBUG:arpav_ppcv.thredds.ncs

In [4]:
for cs in cov_series:
    print(cs.identifier)
    print(cs.data_)
    print("----------")

forecast-tas-absolute-annual-arpa_vfvg-model_ensemble-rcp26-winter-main-t40u9vxjnfs1-*-*-no_smoothing
time
1976-02-15 12:00:00+00:00           2.008203
1977-02-14 17:57:04.390000+00:00    2.386957
1978-02-14 23:54:08.780000+00:00    1.756158
1979-02-15 05:51:13.171000+00:00    2.293268
1980-02-15 11:48:17.561000+00:00    2.184869
                                      ...   
2095-02-15 00:11:42.439000+00:00    3.014734
2096-02-15 06:08:46.829000+00:00    3.708032
2097-02-14 12:05:51.220000+00:00    3.086969
2098-02-14 18:02:55.610000+00:00    3.397485
2099-02-15 00:00:00+00:00           3.743371
Name: forecast-tas-absolute-annual-arpa_vfvg-model_ensemble-rcp26-winter-main-t40u9vxjnfs1-*-*-no_smoothing, Length: 124, dtype: float64
----------
forecast-tas-absolute-annual-arpa_vfvg-model_ensemble-rcp26-winter-main-t40u9vxjnfs1-*-*-moving_average_11_years
time
1976-02-15 12:00:00+00:00          NaN
1977-02-14 17:57:04.390000+00:00   NaN
1978-02-14 23:54:08.780000+00:00   NaN
1979-02-15 05:5

In [5]:
cov_series[0].data_

time
1976-02-15 12:00:00+00:00           2.008203
1977-02-14 17:57:04.390000+00:00    2.386957
1978-02-14 23:54:08.780000+00:00    1.756158
1979-02-15 05:51:13.171000+00:00    2.293268
1980-02-15 11:48:17.561000+00:00    2.184869
                                      ...   
2095-02-15 00:11:42.439000+00:00    3.014734
2096-02-15 06:08:46.829000+00:00    3.708032
2097-02-14 12:05:51.220000+00:00    3.086969
2098-02-14 18:02:55.610000+00:00    3.397485
2099-02-15 00:00:00+00:00           3.743371
Name: forecast-tas-absolute-annual-arpa_vfvg-model_ensemble-rcp26-winter-main-t40u9vxjnfs1-*-*-no_smoothing, Length: 124, dtype: float64

In [6]:
df = cov_series[0].data_.to_frame()

In [17]:
df.columns

Index(['forecast-tas-absolute-annual-arpa_vfvg-model_ensemble-rcp26-winter-main-t40u9vxjnfs1-*-*-no_smoothing'], dtype='object')

In [19]:
cov_series[0].identifier

'forecast-tas-absolute-annual-arpa_vfvg-model_ensemble-rcp26-winter-main-t40u9vxjnfs1-*-*-no_smoothing'

In [21]:
df[cov_series[0].identifier].rolling(center=True, window=11).mean()

time
1976-02-15 12:00:00+00:00          NaN
1977-02-14 17:57:04.390000+00:00   NaN
1978-02-14 23:54:08.780000+00:00   NaN
1979-02-15 05:51:13.171000+00:00   NaN
1980-02-15 11:48:17.561000+00:00   NaN
                                    ..
2095-02-15 00:11:42.439000+00:00   NaN
2096-02-15 06:08:46.829000+00:00   NaN
2097-02-14 12:05:51.220000+00:00   NaN
2098-02-14 18:02:55.610000+00:00   NaN
2099-02-15 00:00:00+00:00          NaN
Name: forecast-tas-absolute-annual-arpa_vfvg-model_ensemble-rcp26-winter-main-t40u9vxjnfs1-*-*-no_smoothing, Length: 124, dtype: float64

In [ ]:
[cs.identifier for cs in cov_series]

In [ ]:
[(cs.identifier, cs.data_.index.dtype) for cs in cov_series]

In [ ]:
coverage_series, observation_series = operations.get_forecast_coverage_time_series(
    settings=settings,
    session=session,
    http_client=None,
    coverage=forecast_coverage,
    point_geom=None,
    temporal_range=None,
    coverage_smoothing_strategies=[],
    observation_smoothing_strategies=[],
    include_coverage_data=True,
    include_observation_data=True,
    include_coverage_uncertainty=True,
    include_coverage_related_models=True
)